# 构建卷积神经网络

* 卷积网络中的输入和层与传统神经网路有点区别，需重新设计，训练模块基本一致

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# 首先读取数据

* 分别构建训练集和测试集（验证集）
* Dataloader 来迭代取数据

In [15]:
# 定义超参数
input_size = 28          # 图像的总尺寸 28 * 28
num_classes = 10         # 标签的种类数
num_epochs = 3           # 训练的总循环周期
batch_size = 64          # 一个批次的大小，64 张图片

In [16]:
# 数据划分
# 训练集
train_dataset = datasets.MNIST(
    root='../../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
# 测试集
test_dataset = datasets.MNIST(
    root='../../data',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

# 构建 batch 数据
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
)

# 卷积网络模块构建

* 一般卷积层、relu层、池化层可以写成一个套餐
* 注意卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(             # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,                  # 灰度图
                out_channels=16,                # 要得到几个特征图
                kernel_size=(5, 5),             # 卷积核大小
                stride=(1, 1),                  # 步长
                padding=2,                      # 边缘填充
            ),                                  # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                          # relu 层
            nn.MaxPool2d(kernel_size=2),        # 池化层 输出结果为 (16, 14, 14)
        )
        self.conv2 = nn.Sequential(             # 下一个套餐的输入
            nn.Conv2d(16, 32, 5, 1, 2),         # 输出  (32, 14, 14)
            nn.ReLU(),                          # relu 层
            nn.MaxPool2d(2),                    # 输出层 （32, 7, 7）
        )
        self.out = nn.Linear(32 * 7 * 7, 10)    # 全连接层得到的结果

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)               # flatten 操作，结果为 (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

# 准确率作为评估标准

In [18]:
def accuracy(prediction, labels):
    pred = torch.max(prediction.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

# 训练网络模型

In [19]:
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 开始训练循环
for epoch in range(num_epochs):
    # 当前 epoch 的结果保存下来
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for data, target in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{:5d} / {} ({:.0f}%)]\t损失: {:.6f}\t训练集正确率: {:.2f}%'.format(
                epoch,
                batch_idx * batch_size,
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]
            ))


当前epoch: 0 [    0 / 60000 (0%)]	损失: 2.307749	训练集正确率: 17.19%
当前epoch: 0 [ 6400 / 60000 (11%)]	损失: 0.355683	训练集正确率: 77.51%
当前epoch: 0 [12800 / 60000 (21%)]	损失: 0.219310	训练集正确率: 85.47%
当前epoch: 0 [19200 / 60000 (32%)]	损失: 0.187387	训练集正确率: 88.76%
当前epoch: 0 [25600 / 60000 (43%)]	损失: 0.159981	训练集正确率: 90.65%
当前epoch: 0 [32000 / 60000 (53%)]	损失: 0.163418	训练集正确率: 91.86%
当前epoch: 0 [38400 / 60000 (64%)]	损失: 0.106917	训练集正确率: 92.75%
当前epoch: 0 [44800 / 60000 (75%)]	损失: 0.073096	训练集正确率: 93.39%
当前epoch: 0 [51200 / 60000 (85%)]	损失: 0.091262	训练集正确率: 93.90%
当前epoch: 0 [57600 / 60000 (96%)]	损失: 0.052043	训练集正确率: 94.28%
当前epoch: 1 [    0 / 60000 (0%)]	损失: 0.049952	训练集正确率: 96.88%
当前epoch: 1 [ 6400 / 60000 (11%)]	损失: 0.042179	训练集正确率: 98.33%
当前epoch: 1 [12800 / 60000 (21%)]	损失: 0.108506	训练集正确率: 98.19%
当前epoch: 1 [19200 / 60000 (32%)]	损失: 0.345055	训练集正确率: 98.03%
当前epoch: 1 [25600 / 60000 (43%)]	损失: 0.027139	训练集正确率: 98.08%
当前epoch: 1 [32000 / 60000 (53%)]	损失: 0.010878	训练集正确率: 98.09%
当前epoch: 1 [38400 / 60000 